# Import data from Smoothcomp 
https://svenskjudo.smoothcomp.com

## Requires
* conda install requests
* conda install pandas
* conda install beautifulsoup4

In [74]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re


## Identify all pages for an event

In [75]:
elbilar_300000kr_5000mil_2021_2025 = ("https://www.bytbil.com/bil?"
            "PriceRange.To=300000&ModelYearRange.From=2021&ModelYearRange.To=2025"
            "&Fuels%5B0%5D=El&&MilageRange.To=5000&OnlyNew=False&OnlyWarrantyProgram=False"
            "&OnlyEnvironmentFriendly=False&OnlyFourWheelDrive=False"
            "&OnlyReducedPrice=False&OnlyDeductibleVAT=False&OnlyIsAuction=False"
            "&OnlyAuthorizedDealers=False&OnlyHasImage=False"
            "&OnlyHasVideo=False&OnlyHasCarfaxReport=False&OnlyNoBonusMalus=False")
elbilar_skane_400000kr_5000mil_noleasing = ("https://www.bytbil.com/bil?"
            "VehicleType=bil&Makes=&FreeText=&Regions=Sk%C3%A5ne+l%C3%A4n&PriceRange.From=&PriceRange.To=400000"
            "&ModelYearRange.From=&ModelYearRange.To=&MilageRange.From=&MilageRange.To=5000&BodyTypes=&Fuels=El"
            "&Gearboxes=&EnginePowerRange.From=&EnginePowerRange.To=&ShowLeasingOffers=False&ShowImportedOffers="
            "&ElectricRangeRange.From=&ElectricRangeRange.To=&SortParams.SortField=publishedDate&SortParams.IsAscending=False")

In [76]:
def get_items_from_url(page_num):
    url = elbilar_skane_400000kr_5000mil_noleasing + f"&Page={page_num}&IgnoreSortFiltering=False"
    r = requests.get(url)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'html.parser')
    items = soup.select("ul > li")
    return items

In [77]:
def parse_page(items):
    rows = []
    for li in items:
        desc_div = li.select_one("div.description.uk-width-2-3.uk-width-medium-3-4.uk-width-xlarge-4-5")
        if not desc_div:
            continue
        h3a = desc_div.select_one("h3 a")
        if not h3a:
            continue
        title = h3a.get_text(strip=True)
        link = "https://www.bytbil.com" + h3a['href']
        
        p = desc_div.find_next("p")
        meta = p.get_text(separator=" | ", strip=True) if p else ""
        p = desc_div.find_next("p")
        year = mileage = location = p.get_text()
        if p:
            parts = [t.strip().replace('\xa0', ' ') for t in p.contents if isinstance(t, str) and t.strip()]
    
            if len(parts) > 0: year = parts[0]
            if len(parts) > 1: mileage = parts[1]
            if len(parts) > 2: location = parts[2]
        
        price_span = li.select_one("div span.car-price-details span.car-price-main")
        price = price_span.get_text(strip=True) if price_span else ""
        
        rows.append({
            "Titel": title,
            "Länk": link,
            "Årsmodell": year,
            "Miltal": mileage,
            "Ort": location,
            "Pris": price
        })
    return rows

In [78]:
def scrape_all(max_pages=10, delay=1.0):
    all_rows = []
    for page in range(1, max_pages + 1):
        print(f"Hämtar sida {page}...")
        items = get_items_from_url(page);
        rows = parse_page(items)
        if not rows:
            print("Slut på data.")
            break
        all_rows.extend(rows)
        time.sleep(delay)
    return all_rows

In [79]:
data = scrape_all(max_pages=10)


Hämtar sida 1...
Hämtar sida 2...
Hämtar sida 3...
Hämtar sida 4...
Hämtar sida 5...
Hämtar sida 6...
Hämtar sida 7...
Hämtar sida 8...
Hämtar sida 9...
Hämtar sida 10...
Slut på data.


In [80]:
df = pd.DataFrame(data)
# df.to_csv("bytbil_elbilar.csv", index=False, encoding="utf-8-sig")
df.to_excel("bytbil_elbilar.xlsx", sheet_name="Elbilar", index=False)
df

,Titel,Länk,Årsmodell,Miltal,Ort,Pris
0,Volkswagen ID.4 Pro Performance 77kwh 204hk,https://www.bytbil.com/skane-lan/personbil-id-...,2023,3 053 mil,MALMÖ,344 900 kr
1,"Kia E-Niro e- 64 kWh, 204hk Adv",https://www.bytbil.com/skane-lan/personbil-e-n...,2022,4 925 mil,ESLÖV,249 000 kr
2,Nissan Leaf 39kWh ACENTA Backkamera 150hk Blue...,https://www.bytbil.com/skane-lan/personbil-lea...,2023,807 mil,Svedala,189 900 kr
3,Fiat 500 e 42 kWh \ 32 mil räckvidd \ 1.951kr/...,https://www.bytbil.com/skane-lan/personbil-500...,2021,2 923 mil,Klippan,179 000 kr
4,Volvo C40 Recharge Single Motor Plus,https://www.bytbil.com/skane-lan/personbil-c40...,2023,4 395 mil,LUND,359 500 kr
...,...,...,...,...,...,...
205,Citroën e-Berlingo Shine M 50kWh Business Lease,https://www.bytbil.com/skane-lan/personbil-e-b...,2023,0 mil,ÄNGELHOLM,9 124 kr
206,"Kia E-Niro 64 kWh 204hk Advance plus Tech, Tak...",https://www.bytbil.com/skane-lan/personbil-e-n...,2019,1 500 mil,KRISTIANSTAD,259 000 kr
207,JAC e-JS4 66 kWh 150hk *Avbetalning/Leasing*,https://www.bytbil.com/skane-lan/personbil-e-j...,2023,3 100 mil,LUND,319 900 kr
208,DS 3 Crossback E-TENSE EXCLUSIVE 50kWh,https://www.bytbil.com/skane-lan/personbil-3-c...,2022,1 346 mil,MALMÖ,289 900 kr
